In [1]:
import numpy as np
import pandas as pd
from hmmlearn import hmm
import warnings
from constants import *
import math
import os
import random
import matplotlib.pyplot as plt
import dill
from functools import reduce

In [7]:
df_rnaseq = pd.read_csv('AdjustedRPKMOutput/RNASeq_chrE_mockT0.txt',sep='\t')
df_rpf = pd.read_csv('AdjustedRPKMOutput/RPF_chrE_mockT0.txt',sep='\t')
df_TE = reduce(lambda left,right: pd.merge(left,right,on=['AccNum','GeneName']), [df_rnaseq,df_rpf])
df_TE.head()

,AccNum,GeneName,cdReads0_x,cdRPKM0_x,cdReads1_x,cdRPKM1_x,cdReads2_x,cdRPKM2_x,cdReads3_x,cdRPKM3_x,...,cdReads0_y,cdRPKM0_y,cdReads1_y,cdRPKM1_y,cdReads2_y,cdRPKM2_y,cdReads3_y,cdRPKM3_y,cdReads4_y,cdRPKM4_y
0,NM_017847,ODR4,50.0,6.672930,62.0,5.568612,71.0,5.308751,30.0,2.859319,...,86.0,13.722243,62.0,13.189469,49.0,14.172717,37.0,9.403085,39.0,10.487249
1,NM_001143986,TLE6,1.0,0.105975,6.0,0.427921,3.0,0.178120,3.0,0.227049,...,2.0,0.253404,2.0,0.337849,2.0,0.459350,1.0,0.201802,1.0,0.213527
2,NM_001003803,ATP5S,41.0,11.526251,52.0,9.838214,52.0,8.190209,19.0,3.814631,...,99.0,33.275110,83.0,37.193868,71.0,43.258655,46.0,24.625447,26.0,14.727463
3,NM_001003800,BICD2,789.0,55.970762,1155.0,55.141030,1444.0,57.390343,706.0,35.767093,...,512.0,43.424443,389.0,43.986848,284.0,43.662942,257.0,34.716778,230.0,32.874747
4,NM_016649,ESF1,22.0,1.567982,26.0,1.247097,10.0,0.399306,9.0,0.458095,...,84.0,7.157770,52.0,5.907596,41.0,6.333047,26.0,3.528692,26.0,3.733723


In [8]:
df_TE[['cdRPKM0_x']] = df_TE[['cdRPKM0_y']].div(df_TE['cdRPKM0_x'].values,axis=0)
df_TE[['cdRPKM1_x']] = df_TE[['cdRPKM1_y']].div(df_TE['cdRPKM1_x'].values,axis=0)
df_TE[['cdRPKM2_x']] = df_TE[['cdRPKM2_y']].div(df_TE['cdRPKM2_x'].values,axis=0)
df_TE[['cdRPKM3_x']] = df_TE[['cdRPKM3_y']].div(df_TE['cdRPKM3_x'].values,axis=0)
df_TE[['cdRPKM4_x']] = df_TE[['cdRPKM4_y']].div(df_TE['cdRPKM4_x'].values,axis=0)
for i in range(0,5):
    df_TE.rename(columns={'cdRPKM'+str(i)+'_x':'TE'+str(i)}, inplace=True)


In [9]:
df_TE = df_TE[['AccNum','GeneName','TE0','TE1','TE2','TE3','TE4']]

In [10]:
df_TE.head()

,AccNum,GeneName,TE0,TE1,TE2,TE3,TE4
0,NM_017847,ODR4,2.056404,2.368538,2.669690,3.288575,3.380689
1,NM_001143986,TLE6,2.391168,0.789513,2.578884,0.888804,1.473634
2,NM_001003803,ATP5S,2.886898,3.780551,5.281752,6.455525,5.224701
3,NM_001003800,BICD2,0.775842,0.797715,0.760806,0.970635,1.151537
4,NM_016649,ESF1,4.564957,4.737076,15.860135,7.702969,5.747171


In [11]:
df_TE.to_csv('AdjustedRPKMOutput/TE_chrE_mockT0.txt',sep='\t',index=False)

In [13]:
df_new = df_TE.nlargest(20, ['TE0','TE1','TE2','TE3','TE4'])